<a href="https://colab.research.google.com/github/gayashan4lk/langchain-for-app-development/blob/main/langchain_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Legacy memory management

These are different memory types for managing conversation history in LangChain:

- ConversationBufferMemory: Stores all messages in full
- ConversationBufferWindowMemory: Keeps only the last N messages
- ConversationTokenBufferMemory: Keeps messages up to a token limit
- ConversationSummaryMemory: Summarizes older messages to save space

The current (langchain v1.0) LangChain/LangGraph docs recommend using **state-based memory** with strategies like **trimming, deleting, or summarizing messages** instead of these legacy memory classes. See the short-term memory guides for modern approaches.

In [3]:
import os
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [7]:
!pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 6.3 MB/s eta 0:00:00


# Short-term memory

In [8]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_openai import ChatOpenAI

gpt_5_nano = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.1,
    max_tokens=4000,
    timeout=30
)

agent = create_agent(
    model=gpt_5_nano,
    checkpointer=InMemorySaver()
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},
)

In [39]:
def print_conversation(result):
  for message in result['messages']:
    print(f"{type(message).__name__}: {message.content}\n")

In [9]:
result

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='9a96e62a-75a9-42ce-b9c8-20842a04f612'),
  AIMessage(content='Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 572, 'prompt_tokens': 13, 'total_tokens': 585, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CcVJMPr0iyQtUVmpVRvqvoWw9duap', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--53996340-a05b-4f58-9fd1-e5521c939994-0', usage_meta

In [10]:
type(result)

dict

In [15]:
human_message_content = result['messages'][0].content
print(human_message_content)

Hi! My name is Bob.


In [11]:
ai_message_content = result['messages'][1].content
print(ai_message_content)

Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.


In [33]:
print_conversation(result)

HumanMessage: Hi! My name is Bob.
AIMessage: Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.


In [18]:
conversation2 = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 1+1 ?"}]},
    {"configurable": {"thread_id": "1"}},
)

In [21]:
conversation2

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='9a96e62a-75a9-42ce-b9c8-20842a04f612'),
  AIMessage(content='Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 572, 'prompt_tokens': 13, 'total_tokens': 585, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CcVJMPr0iyQtUVmpVRvqvoWw9duap', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--53996340-a05b-4f58-9fd1-e5521c939994-0', usage_meta

In [34]:
print_conversation(conversation2)

HumanMessage: Hi! My name is Bob.
AIMessage: Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.
HumanMessage: what is 1+1.
AIMessage: 2

Want a quick explanation or more math help?
HumanMessage: what is 1+1 ?
AIMessage: 2. In base-10 arithmetic, 1 + 1 = 2. Want a quick explanation or more math help?
HumanMessage: I live in Australia
AIMessage: Nice to meet you, Bob. Since you’re in Australia, I can tailor things to your context. A few quick options:
- Time and weather: tell me your city and I can share the current time and typical climate.
- Units and currency: we use metric (Celsius, kilometers) and AUD.
- Australian English: I can help with phrases and slang.
- Everyday stuff: driving on the left, road rules, common tips.
- Travel and geography: info about regions, distances, or famous places.

What would you like to focus on? If y

In [24]:
conversation3 = agent.invoke(
    {"messages": [{"role": "user", "content": "I live in Australia"}]},
    {"configurable": {"thread_id": "1"}},
)

In [35]:
print_conversation(conversation3)

HumanMessage: Hi! My name is Bob.
AIMessage: Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.
HumanMessage: what is 1+1.
AIMessage: 2

Want a quick explanation or more math help?
HumanMessage: what is 1+1 ?
AIMessage: 2. In base-10 arithmetic, 1 + 1 = 2. Want a quick explanation or more math help?
HumanMessage: I live in Australia
AIMessage: Nice to meet you, Bob. Since you’re in Australia, I can tailor things to your context. A few quick options:
- Time and weather: tell me your city and I can share the current time and typical climate.
- Units and currency: we use metric (Celsius, kilometers) and AUD.
- Australian English: I can help with phrases and slang.
- Everyday stuff: driving on the left, road rules, common tips.
- Travel and geography: info about regions, distances, or famous places.

What would you like to focus on? If y

In [26]:
conversation4 = agent.invoke(
    {"messages": [{"role": "user", "content": "I live in Sydney"}]},
    {"configurable": {"thread_id": "1"}},
)

In [36]:
print_conversation(conversation4)

HumanMessage: Hi! My name is Bob.
AIMessage: Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.
HumanMessage: what is 1+1.
AIMessage: 2

Want a quick explanation or more math help?
HumanMessage: what is 1+1 ?
AIMessage: 2. In base-10 arithmetic, 1 + 1 = 2. Want a quick explanation or more math help?
HumanMessage: I live in Australia
AIMessage: Nice to meet you, Bob. Since you’re in Australia, I can tailor things to your context. A few quick options:
- Time and weather: tell me your city and I can share the current time and typical climate.
- Units and currency: we use metric (Celsius, kilometers) and AUD.
- Australian English: I can help with phrases and slang.
- Everyday stuff: driving on the left, road rules, common tips.
- Travel and geography: info about regions, distances, or famous places.

What would you like to focus on? If y

In [37]:
conversation5 = agent.invoke(
    {"messages": [{"role": "user", "content": "Who is the president of Australia"}]},
    {"configurable": {"thread_id": "1"}},
)

In [40]:
print_conversation(conversation5)

HumanMessage: Hi! My name is Bob.

AIMessage: Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.

HumanMessage: what is 1+1.

AIMessage: 2

Want a quick explanation or more math help?

HumanMessage: what is 1+1 ?

AIMessage: 2. In base-10 arithmetic, 1 + 1 = 2. Want a quick explanation or more math help?

HumanMessage: I live in Australia

AIMessage: Nice to meet you, Bob. Since you’re in Australia, I can tailor things to your context. A few quick options:
- Time and weather: tell me your city and I can share the current time and typical climate.
- Units and currency: we use metric (Celsius, kilometers) and AUD.
- Australian English: I can help with phrases and slang.
- Everyday stuff: driving on the left, road rules, common tips.
- Travel and geography: info about regions, distances, or famous places.

What would you like to focus o

In [41]:
conversation6 = agent.invoke(
    {"messages": [{"role": "user", "content": "What are the last 5 prime ministers of Australia"}]},
    {"configurable": {"thread_id": "1"}},
)

In [43]:
print_conversation(conversation6)

HumanMessage: Hi! My name is Bob.

AIMessage: Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.

HumanMessage: what is 1+1.

AIMessage: 2

Want a quick explanation or more math help?

HumanMessage: what is 1+1 ?

AIMessage: 2. In base-10 arithmetic, 1 + 1 = 2. Want a quick explanation or more math help?

HumanMessage: I live in Australia

AIMessage: Nice to meet you, Bob. Since you’re in Australia, I can tailor things to your context. A few quick options:
- Time and weather: tell me your city and I can share the current time and typical climate.
- Units and currency: we use metric (Celsius, kilometers) and AUD.
- Australian English: I can help with phrases and slang.
- Everyday stuff: driving on the left, road rules, common tips.
- Travel and geography: info about regions, distances, or famous places.

What would you like to focus o

In [45]:
conversation7 = agent.invoke(
    {"messages": [{"role": "user", "content": "What is my name and where do I live?"}]},
    {"configurable": {"thread_id": "1"}},
)

In [46]:
print_conversation(conversation7)

HumanMessage: Hi! My name is Bob.

AIMessage: Hi Bob! Nice to meet you. How can I help you today? I can assist with writing, learning, planning, coding, brainstorming, and more. Tell me what you’d like to work on, or drop a topic you’re curious about.

HumanMessage: what is 1+1.

AIMessage: 2

Want a quick explanation or more math help?

HumanMessage: what is 1+1 ?

AIMessage: 2. In base-10 arithmetic, 1 + 1 = 2. Want a quick explanation or more math help?

HumanMessage: I live in Australia

AIMessage: Nice to meet you, Bob. Since you’re in Australia, I can tailor things to your context. A few quick options:
- Time and weather: tell me your city and I can share the current time and typical climate.
- Units and currency: we use metric (Celsius, kilometers) and AUD.
- Australian English: I can help with phrases and slang.
- Everyday stuff: driving on the left, road rules, common tips.
- Travel and geography: info about regions, distances, or famous places.

What would you like to focus o